In [1]:
import os
import obspy 
import matplotlib.pyplot as plt
import numpy as np
from obspy.core.inventory import Inventory, Network, Station, Channel, Site 
from obspy import UTCDateTime, read, read_inventory

In [2]:
def plot_particle_motion_2D(axis, trace1, trace2):
    '''
    function to plot a subfigure in a particle motion figure
    param axis: figure axis to use for plotting 
    type axis: figure axis
    param trace1: 1 component trimmed seismic data
    type trace1: Obspy Trace object
    param trace2: 1 component trimmed seismic data of a different component
    type trace2: Obspy Trace object
    '''
    
    # Plot trace1 against trace2 to visualize particle motion
    axis.plot(trace1.data, trace2.data, "k-", linewidth=0.5)

     # Label the axes with the channel names of trace1 and trace2
    axis.set_ylabel(trace2.stats.channel)
    axis.set_xlabel(trace1.stats.channel)

    # Mark the starting point of the ground motion with a blue star
    axis.plot(trace1[0],trace2[0], marker = "*", markersize=10, color = "b")

def plot_event(def_event, def_st, def_stfreq, def_stcopy_pm, def_NFFT, def_noverlap, def_i, def_Ptime, def_Stime):
    '''
    Function to plot seismic data (waveform and spectrogram in one figure for each component) and particle motion for a given event.
    
    :param def_event: Event information.
    :type def_event: list
    :param def_st: Stream object containing seismic data.
    :type def_st: obspy.core.stream.Stream
    :param def_stfreq: Stream object containing frequency domain seismic data.
    :type def_stfreq: obspy.core.stream.Stream
    :param def_stcopy_pm: Stream object containing seismic data for particle motion.
    :type def_stcopy_pm: obspy.core.stream.Stream
    :param def_NFFT: Number of data points used in each block for the FFT.
    :type def_NFFT: int
    :param def_noverlap: Number of points of overlap between blocks.
    :type def_noverlap: int
    :param def_i: Index of the current event.
    :type def_i: int
    :param def_Ptime: List of P wave arrival times.
    :type def_Ptime: list
    :param def_Stime: List of S wave arrival times.
    :type def_Stime: list
    '''

    # Determine event type based on the fourth element of def_event
    if def_event[3] == '0':
        def_type = 'typeA'
    else:
        def_type = 'typeB'

    print(def_event[3], def_type)

     # Check if the stream is empty and skip plotting if true
    if len(def_st) == 0:
        print(f"Stream is empty for event {def_event}, skipping plot.")
        return
    
    ### Plotting HHZ component

    # Create the figure and two subplots for the seismogram and spectrogram
    fig1, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), constrained_layout=True, sharex=True)

    # Plot the seismogram on ax1
    ax1.plot(def_st[2].times(), def_st[2].data, 'k-', linewidth = 0.8, label='%s.%s.%s.%s' % (def_st[2].stats.network, def_st[2].stats.location, def_st[2].stats.station, def_st[2].stats.channel))
    ax1.axvline(x=10, ymin=0.4, ymax=0.6, color='red', linestyle='-', linewidth = 1, label='P wave') # Add P arrival time
    ax1.legend(fontsize=14)
    ax1.set_title('%s - VT event %s' % (str(def_st[2].stats.starttime - 10)[:-5], def_type), size=16)
    ax1.set_ylabel('Velocity [m/s]', size=14)
    ax1.tick_params(axis='both', which='minor', labelsize = 9)
    ymax = max(def_st[2].data)
    ymin = -ymax
    ax1.set_ylim(ymin, ymax)
    ax1.grid(color='gray', linestyle = '-', linewidth = 0.2)

    # Plot the spectrogram on ax2 - using a suitable vmax for better visualization
    max_values = []

    #ax2.set_title('Spectrogram, window length %s pts' % def_NFFT, size=16)
    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[2].data, NFFT=def_NFFT, Fs=def_stfreq[2].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear')

    max_values.append(Pxx.max())
    vmax = np.percentile(max_values, 95)

    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[2].data, NFFT=def_NFFT, Fs=def_stfreq[2].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear', vmax=vmax*0.1)

    ax2.set_xlabel('Relative time [sec]', size=12)
    ax2.tick_params(axis='both', which='minor', labelsize = 9)
    ax2.set_ylabel('Frequency [Hz]', size=14)
    ax2.set_ylim(0, 30)

    cbar = fig1.colorbar(im, ax=ax2)  # Add colorbar for the spectrogram
    cbar.set_label(r'Power spectral density $[(m/s)^2/Hz]$', size=12)
    
    # Save the figure
    save_name1 = def_event[1] + '_' + str(def_i) + '_' + str(def_type) + '.png'
    fig1.savefig("../figures/seismograms/HHZ/" + save_name1)
    plt.close(fig1)

    ### Plotting HHN component

    # Create the figure and two subplots for the seismogram and spectrogram
    fig2, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), constrained_layout=True, sharex=True)

   # Plot the seismogram on ax1
    ax1.plot(def_st[1].times(), def_st[1].data, 'k-', linewidth = 0.8, label='%s.%s.%s.%s' % (def_st[1].stats.network, def_st[1].stats.location, def_st[1].stats.station, def_st[1].stats.channel))
    if def_Stime[def_i] is not None: # Add S arrival time if exists
        p_time = UTCDateTime(def_Ptime[def_i])
        s_time = UTCDateTime(def_Stime[def_i])
        time_difference = s_time - p_time  # This will be in seconds
        ax1.axvline(x = 10 + time_difference, ymin=0.4, ymax=0.6, color='blue', linestyle='-', linewidth = 1, label='S wave')
    else:
        print(f"No S phase for P time at index {def_i}: {def_Ptime[def_i]}")
    ax1.legend(fontsize=14)
    ax1.set_title('%s - VT event %s' % (str(def_st[1].stats.starttime - 10)[:-5], def_type), size=16)
    ax1.set_ylabel('Velocity [m/s]', size=14)
    ax1.tick_params(axis='both', which='minor', labelsize = 9)
    ymax = max(def_st[1].data)
    ymin = -ymax
    ax1.set_ylim(ymin, ymax)
    ax1.grid(color='gray', linestyle = '-', linewidth = 0.2)

    # Plot the spectrogram on ax2 - using a suitable vmax for better visualization
    max_values = []

    # ax2.set_title('Spectrogram, window length %s pts' % def_NFFT, size=16)
    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[1].data, NFFT=def_NFFT, Fs=def_stfreq[1].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear')

    max_values.append(Pxx.max())
    vmax = np.percentile(max_values, 95)

    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[1].data, NFFT=def_NFFT, Fs=def_stfreq[1].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear', vmax=vmax*0.1)

    ax2.set_xlabel('Relative time [sec]', size=16)
    ax2.tick_params(axis='both', which='minor', labelsize = 5)
    ax2.set_ylabel('Frequency [Hz]', size=14)
    ax2.set_ylim(0, 30)

    cbar = fig2.colorbar(im, ax=ax2) # Add colorbar for the spectrogram
    cbar.set_label(r'Power spectral density $[(m/s)^2/Hz]$', size=12)

    # Save the figure
    save_name2 = def_event[1] + '_' + str(def_i) + '_' + str(def_type) + '.png'
    fig2.savefig("../figures/seismograms/HHN/" + save_name2)
    plt.close(fig2)
    
    ### Plotting HHN component

    # Create the figure and two subplots for the seismogram and spectrogram
    fig3, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), constrained_layout=True, sharex=True)

    # Plot the seismogram on ax1
    ax1.plot(def_st[0].times(), def_st[0].data, 'k-', linewidth = 0.8, label='%s.%s.%s.%s' % (def_st[0].stats.network, def_st[0].stats.location, def_st[0].stats.station, def_st[0].stats.channel))
    if def_Stime[def_i] is not None:  # Add S arrival time if exists
        p_time = UTCDateTime(def_Ptime[def_i])
        s_time = UTCDateTime(def_Stime[def_i])
        time_difference = s_time - p_time  # This will be in seconds
        ax1.axvline(x = 10 + time_difference, ymin=0.4, ymax=0.6, color='blue', linestyle='-', linewidth = 1, label='S wave')
    else:
        print(f"No S phase for P time at index {def_i}: {def_Ptime[def_i]}")
    ax1.legend(fontsize=14)
    ax1.set_title('%s - VT event %s' % (str(def_st[0].stats.starttime - 10)[:-5], def_type), size=16)
    ax1.set_ylabel('Velocity [m/s]', size=14)
    ax1.tick_params(axis='both', which='minor', labelsize = 9)
    ymax = max(def_st[0].data)
    ymin = -ymax
    ax1.set_ylim(ymin, ymax)
    ax1.grid(color='gray', linestyle = '-', linewidth = 0.2)
        
    # Plot the spectrogram on ax2 - using a suitable vmax for better visualization
    max_values = []

    # ax2.set_title('Spectrogram, window length %s pts' % def_NFFT, size=16)
    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[0].data, NFFT=def_NFFT, Fs=def_stfreq[0].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear')

    max_values.append(Pxx.max())
    vmax = np.percentile(max_values, 95)

    Pxx, freqs, bins, im = ax2.specgram(
        def_stfreq[0].data, NFFT=def_NFFT, Fs=def_stfreq[0].stats.sampling_rate, 
        noverlap=def_noverlap, cmap=plt.cm.viridis, scale='linear', vmax=vmax*0.1)

    ax2.set_xlabel('Relative time [sec]', size=16)
    ax2.tick_params(axis='both', which='minor', labelsize = 5)
    ax2.set_ylabel('Frequency [Hz]', size=14)
    ax2.set_ylim(0, 30)

    cbar = fig3.colorbar(im, ax=ax2) # Add colorbar for the spectrogram
    cbar.set_label(r'Power spectral density $[(m/s)^2/Hz]$', size=12)

    # Save the figure
    save_name3 = def_event[1] + '_' + str(def_i) + '_' + str(def_type) + '.png'
    fig3.savefig("../figures/seismograms/HHE/" + save_name3)
    plt.close(fig3)

    # Plotting particle motion

    if def_stcopy_pm is not None:
        fig4, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
        plot_particle_motion_2D(ax1, def_stcopy_pm[0], def_stcopy_pm[1])
        plot_particle_motion_2D(ax2, def_stcopy_pm[2], def_stcopy_pm[1])
        plot_particle_motion_2D(ax3, def_stcopy_pm[0], def_stcopy_pm[2])

    # Calculate combined limits for x and y axes
        all_x_data = [def_stcopy_pm[0], def_stcopy_pm[2], def_stcopy_pm[0]]
        all_y_data = [def_stcopy_pm[1], def_stcopy_pm[1], def_stcopy_pm[2]]

        x_min = min(min(data) for data in all_x_data)
        x_max = max(max(data) for data in all_x_data)
        y_min = min(min(data) for data in all_y_data)
        y_max = max(max(data) for data in all_y_data)
        padding_x = 0.1 * (x_max - x_min)
        padding_y = 0.1 * (y_max - y_min)
        x_min -= padding_x
        x_max += padding_x
        y_min -= padding_y
        y_max += padding_y

    # Set the same limits for all subplots
        for ax in [ax1, ax2, ax3]:
            ax.set_xlim(x_min, x_max)
            ax.set_ylim(y_min, y_max)
            ax.tick_params(axis='both', which='major', labelsize=14)
            ax.tick_params(axis='both', which='minor', labelsize=12)
            ax.xaxis.label.set_size(16)
            ax.yaxis.label.set_size(16)

        plt.suptitle('%s - VT event %s' % (str(def_st[0].stats.starttime - 3)[:-5], def_type), size=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjusting the rect parameter to make room for the suptitle

    # Save the figure
        save_name4 = def_event[1] + '_' + str(def_i) + '_' + str(def_type) + '.png'
        fig4.savefig("../figures/particlemotion/" + save_name4)
        plt.close(fig4)

In [3]:
### load data

events = np.loadtxt("../catalog/alleventsfinal.txt", dtype = np.str_) # all picks
eventsP = np.array([row for row in events if 'P' in row]) # only P picks = number of events

inv = read_inventory("../DATA/Stations_Reykjanes2021.xml",format='STATIONXML')

In [4]:
file_path = "../catalog/alleventsfinal.txt"

with open(file_path, 'r') as file:
    lines = file.readlines()

events_pairs = []
current_p = None

for line in lines:

    parts = line.split()
    timestamp = parts[1] + "T" + parts[2]
    phase_type = parts[-3]
    event_type = parts[3]

    if phase_type == 'P':
        if current_p is not None:
            events_pairs.append((current_p, None))
        current_p = timestamp
    elif phase_type == 'S':
        if current_p is not None:
            events_pairs.append((current_p, timestamp))
            current_p = None
    
PSpairs = np.array(events_pairs, dtype=object)

Ptime = PSpairs[:, 0]
Stime = PSpairs[:, 1]

In [5]:
# needed variables

i = 0 # for naming figures
NFFT = 256  # length of spectrogram window in sample points
noverlap = 127 # number of sample points that the sliding window overlaps, must be less than NFFT

In [6]:
stZ = read("../DATA/HHZ.D/RR.SAND.00.HHZ.D.2024.001")
stN = read("../DATA/HHN.D/RR.SAND.00.HHN.D.2024.001")
stE = read("../DATA/HHE.D/RR.SAND.00.HHE.D.2024.001")
st = stE + stN + stZ 

day = '001'

for event in eventsP:
    day_new = '0' + event[1][8:10]
   
    if day_new != day: # For processing the data faster
        pathZ = f"../DATA/HHZ.D/RR.SAND.00.HHZ.D.2024.{day_new}"
        pathN = f"../DATA/HHN.D/RR.SAND.00.HHN.D.2024.{day_new}"
        pathE = f"../DATA/HHE.D/RR.SAND.00.HHE.D.2024.{day_new}"

        stZ = read(f"{pathZ}")
        stN = read(f"{pathN}")
        stE = read(f"{pathE}")
        st = stE + stN + stZ  
        
        day = day_new
        
    else: 
        print(day_new, ' = ', day)
        
    if len(stZ) == 0 or len(stN) == 0 or len(stE) == 0:
        print("One of the streams is empty, skipping this event.")
        continue
        
    tstart = event[1] + 'T' + event[2]
    stcopy = st.copy()
    stcopy.trim(UTCDateTime(tstart) - 15, UTCDateTime(tstart) + 25)
    stcopy.detrend('demean')
    stcopy.detrend('linear')
    stcopy.taper(0.1)
    stcopy.remove_response(inventory=inv, output="VEL")
    stcopy_freq = stcopy.copy()
    stcopy.filter("bandpass", freqmin=5.0, freqmax=15.0)
    stcopy.trim(UTCDateTime(tstart) - 10.0, UTCDateTime(tstart) + 20.0)
    stcopy_freq.filter("bandpass", freqmin=1.0, freqmax=20.0)
    stcopy_freq.trim(UTCDateTime(tstart) - 10.0, UTCDateTime(tstart) + 20.0)
    
    stcopy_pm = stcopy.copy()
    stcopy_pm.trim(UTCDateTime(tstart), UTCDateTime(tstart) + 3.0)
    
    plot_event(event, stcopy, stcopy_freq, stcopy_pm, NFFT, noverlap, i, Ptime, Stime)
    
    i = i + 1

    print('Done: ', event[1], 'T', event[2])

001  =  001
0 typeA
Done:  2024-01-01 T 00:31:26.31462
001  =  001
1 typeB
No S phase for P time at index 1: 2024-01-01T00:48:50.89252
No S phase for P time at index 1: 2024-01-01T00:48:50.89252
Done:  2024-01-01 T 00:48:50.89252
001  =  001
1 typeB
No S phase for P time at index 2: 2024-01-01T00:54:38.40124
No S phase for P time at index 2: 2024-01-01T00:54:38.40124
Done:  2024-01-01 T 00:54:38.40124
001  =  001
0 typeA
Done:  2024-01-01 T 00:56:23.14945
001  =  001
0 typeA
Done:  2024-01-01 T 01:02:23.09840
001  =  001
0 typeA
Done:  2024-01-01 T 01:03:03.96540
001  =  001
1 typeB
No S phase for P time at index 6: 2024-01-01T01:08:13.95183
No S phase for P time at index 6: 2024-01-01T01:08:13.95183
Done:  2024-01-01 T 01:08:13.95183
001  =  001
1 typeB
No S phase for P time at index 7: 2024-01-01T01:12:00.15388
No S phase for P time at index 7: 2024-01-01T01:12:00.15388
Done:  2024-01-01 T 01:12:00.15388
001  =  001
1 typeB
No S phase for P time at index 8: 2024-01-01T01:13:53.10993


C:\Users\User\AppData\Local\Temp\ipykernel_14572\3497661579.py:80: UserWarning: Only one segment is calculated since parameter NFFT (=256) >= signal length (=72).
  Pxx, freqs, bins, im = ax2.specgram(
C:\Users\User\AppData\Local\Temp\ipykernel_14572\3497661579.py:87: UserWarning: Only one segment is calculated since parameter NFFT (=256) >= signal length (=72).
  Pxx, freqs, bins, im = ax2.specgram(
C:\Users\User\AppData\Local\Temp\ipykernel_14572\3497661579.py:182: UserWarning: Only one segment is calculated since parameter NFFT (=256) >= signal length (=72).
  Pxx, freqs, bins, im = ax2.specgram(
C:\Users\User\AppData\Local\Temp\ipykernel_14572\3497661579.py:189: UserWarning: Only one segment is calculated since parameter NFFT (=256) >= signal length (=72).
  Pxx, freqs, bins, im = ax2.specgram(


Done:  2024-01-06 T 00:48:30.48854
006  =  006
0 typeA
Done:  2024-01-06 T 00:58:18.64711
006  =  006
0 typeA
Done:  2024-01-06 T 01:15:20.21296
006  =  006
0 typeA
Done:  2024-01-06 T 01:34:20.31284
006  =  006
1 typeB
No S phase for P time at index 414: 2024-01-06T02:05:32.14711
No S phase for P time at index 414: 2024-01-06T02:05:32.14711
Done:  2024-01-06 T 02:05:32.14711
006  =  006
0 typeA
Done:  2024-01-06 T 02:08:22.32863
006  =  006
1 typeB
No S phase for P time at index 416: 2024-01-06T02:33:58.34934
No S phase for P time at index 416: 2024-01-06T02:33:58.34934
Done:  2024-01-06 T 02:33:58.34934
006  =  006
1 typeB
No S phase for P time at index 417: 2024-01-06T02:48:48.94816
No S phase for P time at index 417: 2024-01-06T02:48:48.94816
Done:  2024-01-06 T 02:48:48.94816
006  =  006
1 typeB
No S phase for P time at index 418: 2024-01-06T02:49:07.96619
No S phase for P time at index 418: 2024-01-06T02:49:07.96619
Done:  2024-01-06 T 02:49:07.96619
006  =  006
0 typeA
Done:  20